# Final Project - Random Forests
## Hudson Arney & Ian Golvach

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy.stats
import math
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier # For benchmark testing
from sklearn.metrics import accuracy_score
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
import time
import memory_profiler
import psutil
from tqdm import tqdm

In [ ]:
df = pd.read_csv('Social_Network_Ads.csv')
df.head()

In [ ]:
colors = ['#0000ff', '#00ff00']
cmap = ListedColormap(colors)
plt.scatter(data=df, x='Age', y='EstimatedSalary', c='Purchased', cmap=cmap)
plt.xlabel('Age')
plt.ylabel('Estimated Salary')
plt.title('Age vs Estimated Salary')

legend_labels = ['Not Purchased', 'Purchased']
legend_markers = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='#0000ff', markersize=10),
                  plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='#00ff00', markersize=10)]
plt.legend(legend_markers, legend_labels, loc='upper left')

In [ ]:
df.describe()

In [ ]:
df.info()

___
## Scratch implemntation of RF
Learning sourced from: https://en.wikipedia.org/wiki/Random_forest

Based on the wikipedia article, we will need to modify a standard decision tree model so that
 - Along with the gini coef. split, we select a random subset of the feature to consider when making splits
 - Several decision trees are trained with a 'bagging' approach, training the trees on a random swample w/ replacement of the data.
 
Following this, it makes the most sense that we first implement a method that will handle the sampling and training, as well as the predicting via plurality, and another method that will server as the model itself for the underlying decision trees (that way we can test the bagging concept with other models)
___

In [ ]:
class bagging_trees:
    """Bootstrap aggregates a decision tree, bagging B times"""
    def __init__(self, model, B=5, is_classifier=True):
        self.B = B
        self.model = model
        self.fitted_models = []
        self.is_classifier = is_classifier

    def fit(self, X, y):
        # Create B models fitted on n sets created by randomly sampling with replacement from X and y
        if(X.shape[0] != y.shape[0]):
            raise Exception("X and y do not have the same number of columns")
        for i in range(self.B):
            self.__progress(i)
            bag_X = []
            bag_y = []
            for i in range(X.shape[0]):
                sample_ind = np.random.randint(X.shape[0])
                bag_X.append(X[sample_ind])
                bag_y.append(y[sample_ind])
            new_model = self.model()
            new_model.fit(np.array(bag_X), np.array(bag_y))
            self.fitted_models.append(new_model)
        self.__progress(self.B+1)
            
    def predict(self, X_pred):
        # Return the average of predictions if regression, otherwise return plurality of predictions
        # It may be advantageous to later replace these with np functions
        predictions = []
        for i in range(self.B):
            prediction = self.fitted_models[i].predict(X_pred)
            if np.issubdtype(prediction.dtype, np.number):  # Check if prediction is numeric
                predictions.append(prediction)
        if not predictions:  # Check if predictions list is empty
            raise ValueError("No numeric predictions found")
        
        predictions_array = np.array(predictions)  # Convert to numpy array
        if self.is_classifier:
            return np.ravel(scipy.stats.mode(predictions_array.T, axis=1, keepdims=False)[0])
        else:
            return np.mean(predictions_array.T, axis=1)
        
    def __progress(self, current):
        if(current-1 == self.B):
            print("[=] Sub-model "+str(self.B)+'/'+str(self.B)+' fitted. Fitting complete!')
        else:
            spinner = ' '
            # |/-\
            spinny = current%4
            if(spinny==0):
                spinner = '|'
            elif(spinny==1):
                spinner = '/'
            elif(spinny==2):
                spinner = '-'
            elif(spinny==3):
                spinner = '\\'
            print("["+spinner+"] Sub-model "+str(current+1)+'/'+str(self.B)+' fitting...', end='\r')

        
        

This code is just a proof of concept that the class actually works, which is why train/test splitting was not performed.

In [ ]:
bag_trees = bagging_trees(DecisionTreeClassifier)

In [ ]:
X = np.array(df[['Age','EstimatedSalary']])
y= np.array(df['Purchased'])

In [ ]:
bag_trees.fit(X, y)

In [ ]:
bag_trees.predict(X)

Now we implement a decision tree that picks features at random.

In [ ]:
y/2

In [ ]:
np.unique(y, return_counts=True)

In [ ]:
test_array = [1,3,4,2,5]
test_array.sort()
test_array.remove(2)
test_array

___

In [ ]:
class rdt_node:
    #type(X)==np.ndarray
    """
    Node for random_decision_tree, has a boundary, variable number, and between 0 and 2 children
    Children of this node can either be another node, or a value.
       
    var_num: the variable that is compared at this node
    boundary: the boundary along the variable, <= is left, otherwise right
    left: the left child of the node, or the value of the leaf
    right: the right child of the node, or the value of the leaf
    """
    def __init__(self, var_num=0, boundary=0, left=None, right=None):
        self.var_num = var_num
        self.boundary = boundary
        self.left = left
        self.right = right


class random_decision_tree:
    """
    Decision Tree that follows random forest standards
    
    depth_limit: how deep the forest is allowed to go before stopping
    feature_subset_func: the function used to determine how many feature to use based on number of features
    """
    def __init__(self, depth_limit=5, feature_subset_func=lambda x: math.floor(pow(x,1/2))):
        self.depth_limit = 5
        self.root = rdt_node();
        self.num_features = -1;
        self.feature_subset_func = feature_subset_func
    
    def fit(self, X_train, y_train):
        # Remove duplicate observations
        unique_indices = np.unique(X_train, axis=0, return_index=True)[1]
        X_train_unique = X_train[unique_indices]
        y_train_unique = y_train[unique_indices]
        
        self.num_features = X_train_unique.shape[1]
        self.__fit(X_train_unique, y_train_unique, 0, self.root)

        
        
    def __fit(self, X_train, y_train, depth, node):
        self.num_features = X_train.shape[1]
        gini_result = self.__find_gini(X_train, y_train, self.__array_for_range(X_train.shape[1]))
        if gini_result[0] is None:  # Check if boundary is None
            return
        node.var_num = gini_result[1]
        node.boundary = gini_result[0]
        
        if self.depth_limit == depth or gini_result[2]:
            node.left = scipy.stats.mode(y_train[X_train[:, node.var_num] <= node.boundary], keepdims=False)[0]
            node.right = scipy.stats.mode(y_train[X_train[:, node.var_num] > node.boundary], keepdims=False)[0]
        else:
            node.left = rdt_node()
            node.right = rdt_node()
            self.__fit(X_train[X_train[:, node.var_num] <= node.boundary], y_train[X_train[:, node.var_num] <= node.boundary], depth + 1, node.left)
            self.__fit(X_train[X_train[:, node.var_num] > node.boundary], y_train[X_train[:, node.var_num] > node.boundary], depth + 1, node.right)
            
    def __array_for_range(self, num_for_array):
        # Sometimes the simple solution works
        ret = []
        for i in range(num_for_array):
            ret.append(i)
        return ret
    
    def __find_gini(self, X_train, y_train, features):
        unincluded_features = self.__array_for_range(X_train.shape[1])
        for x in features:
            unincluded_features.remove(x)
        
        boundary_candidates = []
        boundaries = []
        
        for feature_index in features:
            unique_values = np.unique(X_train[:, feature_index])
            
            # Skip features with no variability
            if len(unique_values) == 1:
                continue
            
            if len(unique_values) == 2:
                boundary_candidates.append((np.mean(unique_values), feature_index))
            else:
                for i in range(len(unique_values) - 1):
                    boundary = (unique_values[i] + unique_values[i + 1]) / 2
                    boundary_candidates.append((boundary, feature_index))
        
        if len(boundary_candidates) == 0:
            return (None, None, False)
            # raise Exception("No candidate boundaries found due to lack of variability in features")
        
        for candidate in boundary_candidates:
            boundaries.append((candidate[0], candidate[1], self.__gini_index_for_boundary(X_train, y_train, candidate[0], candidate[1])))
            
        boundaries.sort(key=self.__sorter)
        
        best_boundary = boundaries[0]
        left_leaf = (len(np.unique(y_train[X_train[:, best_boundary[1]] <= best_boundary[0]])) == 1)
        right_leaf = (len(np.unique(y_train[X_train[:, best_boundary[1]] > best_boundary[0]])) == 1)
        is_leaf = left_leaf and right_leaf
        
        return (best_boundary[0], best_boundary[1], is_leaf)


        
        
    def __gini_index_for_boundary(self, X_train, y_train, boundary, feature_num):
        #classification assumed
        
        #calculate proportions of feature_num <= boundary for 1, proportion of feature_num > boundary for 2
        # code pulled from https://stackoverflow.com/questions/67586928/how-can-you-find-what-percentage-of-a-numpy-array-has-some-value-x
        # uniques, counts = np.unique(array, return_counts=True)
        # percentages = dict(zip(uniques, counts * 100 / len(array)))
    
        # r1 is the subset of y masked by X[:,feature_num]<= or > the boundary, because we use it to find the proportions
        R_1 = y_train[X_train[:,feature_num]<=boundary]
        N_1 = R_1.shape[0]
        proportions_1 = np.unique(R_1,return_counts=True)[1]/N_1
        
        R_2 = y_train[X_train[:,feature_num]>boundary]
        N_2 = R_2.shape[0]
        proportions_2 = np.unique(R_2,return_counts=True)[1]/N_2
        
        N = X_train.shape[0]
        
        #compute ginis
        g_1 = 0
        for prop in proportions_1:
            g_1 = g_1 + prop*(1-prop)
            
        g_2 = 0
        for prop in proportions_2:
            g_2 = g_2 + prop*(1-prop)
            
        #return weighted average of ginis
        return ((N_1/N)*(g_1))+((N_2/N)*(g_2))
        
    def __sorter(self, the_tuple):
        return the_tuple[2]
        
    def predict(self, X_pred):
        return np.array([self.__predict(x) for x in X_pred])
    
    def __predict(self, x_pred):
        # For any vector of features given, transverses the tree in order to return the value
        # x_pred is a vector
        tolkein = self.root # could replace this with 'walker' or something else
        while(type(tolkein)==type(self.root)):
            if(x_pred[tolkein.var_num] <= tolkein.boundary):
                tolkein = tolkein.left
            else:
                tolkein = tolkein.right
        return tolkein
    

In [ ]:
rdt = random_decision_tree()
rdt.fit(X, y)
predicted = rdt.predict(X)
print(accuracy_score(y,predicted))

In [ ]:
rdt.root.var_num

___
Put it all together

In [ ]:
random_forest = bagging_trees(random_decision_tree, 100)
random_forest.fit(X, y)
prediction = random_forest.predict(X)
print(accuracy_score(y,prediction))

In [ ]:
print(X.shape)
print(y.shape)
print(predicted.shape)

___
### Plot the Decision Boundaries of the Random Forests

In [ ]:
colors = ['#87cefa', '#ff0000']
cmap = ListedColormap(colors)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=prediction, cmap=cmap, label='Predicted Labels')
plt.xlabel('Age')
plt.ylabel('Estimated Salary')
plt.title('Predicted Labels')

legend_labels = ['Not Purchased', 'Purchased']
legend_markers = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='#87cefa', markersize=10),
                  plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='#ff0000', markersize=10)]
plt.legend(legend_markers, legend_labels, loc='upper left')

In [ ]:
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 5),
                     np.arange(y_min, y_max, 5))

Z = rdt.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Plot the decision boundary
plt.contourf(xx, yy, Z, alpha=0.4)
plt.scatter(X[:, 0], X[:, 1], c=y, s=20, edgecolor='k')

plt.xlabel('Age')
plt.ylabel('Estimated Salary')
plt.title('Decision Boundary of Random Forest Classifier')
plt.show()

____

### Benchmarking

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train and evaluate Decision Tree
dt_classifier = DecisionTreeClassifier()
dt_classifier.fit(X_train, y_train)
dt_prediction = dt_classifier.predict(X_test)
dt_accuracy = accuracy_score(y_test, dt_prediction)
print(f'Decision Tree Accuracy: {dt_accuracy}')

# Train and evaluate custom Random Forest
random_forest = bagging_trees(random_decision_tree, 100)
random_forest.fit(X_train, y_train)
rf_prediction = random_forest.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_prediction)
print(f'Custom Random Forest Accuracy: {rf_accuracy}')

In [ ]:
def measure_memory(func):
    def wrapper(*args, **kwargs):
        start_memory = memory_profiler.memory_usage()[0]
        result = func(*args, **kwargs)
        end_memory = memory_profiler.memory_usage()[0]
        memory_consumption = max(0, end_memory - start_memory)
        return result, memory_consumption
    return wrapper

# Train and evaluate Decision Tree
@measure_memory
def evaluate_decision_tree():
    start_time = time.time()
    dt_classifier = DecisionTreeClassifier()
    dt_classifier.fit(X_train, y_train)
    dt_prediction = dt_classifier.predict(X_test)
    dt_accuracy = accuracy_score(y_test, dt_prediction)
    dt_runtime = time.time() - start_time
    return dt_accuracy, dt_runtime

dt_result, dt_memory_consumption = evaluate_decision_tree()
print(f'Decision Tree Accuracy: {dt_result[0]}')
print(f'Decision Tree Runtime: {dt_result[1]} seconds')
print(f'Decision Tree Memory Consumption: {dt_memory_consumption} MB')


In [ ]:
# Train and evaluate custom Random Forest
@measure_memory
def evaluate_random_forest():
    start_time = time.time()
    random_forest = bagging_trees(random_decision_tree, 100)
    random_forest.fit(X_train, y_train)
    rf_prediction = random_forest.predict(X_test)
    rf_accuracy = accuracy_score(y_test, rf_prediction)
    rf_runtime = time.time() - start_time
    return rf_accuracy, rf_runtime

rf_result, rf_memory_consumption = evaluate_random_forest()
print(f'Custom Random Forest Accuracy: {rf_result[0]}')
print(f'Custom Random Forest Runtime: {rf_result[1]} seconds')
print(f'Custom Random Forest Memory Consumption: {rf_memory_consumption} MB')

In [ ]:
@measure_memory
def evaluate_sklearn_random_forest():
    start_time = time.time()
    rf_classifier = RandomForestClassifier(n_estimators=100)
    rf_classifier.fit(X_train, y_train)
    rf_prediction = rf_classifier.predict(X_test)
    rf_accuracy = accuracy_score(y_test, rf_prediction)
    rf_runtime = time.time() - start_time
    return rf_accuracy, rf_runtime

sklearn_rf_result, sklearn_rf_memory_consumption = evaluate_sklearn_random_forest()
print(f'Scikit-learn Random Forest Accuracy: {sklearn_rf_result[0]}')
print(f'Scikit-learn Random Forest Runtime: {sklearn_rf_result[1]} seconds')
print(f'Scikit-learn Random Forest Memory Consumption: {sklearn_rf_memory_consumption} MB')


In [ ]:
labels = ['Decision Tree', 'Custom Random Forest', 'Scikit-learn Random Forest']
memory_consumptions = [dt_memory_consumption, rf_memory_consumption, sklearn_rf_memory_consumption]

plt.bar(labels, memory_consumptions, color=['red', 'blue', 'green'])
plt.xticks(rotation=45)
plt.xlabel('Classifier')
plt.ylabel('Memory Consumption (MB)')
plt.title('Memory Consumption Comparison')
plt.show()

In [ ]:
runtimes = [dt_result[1], rf_result[1], sklearn_rf_result[1]]

plt.bar(labels, runtimes, color=['red', 'blue', 'green'])
plt.xlabel('Classifier')
plt.ylabel('Runtime (seconds)')
plt.title('Runtime Comparison')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Function to measure CPU usage
def measure_cpu():
    return psutil.cpu_percent()

@measure_memory
# Function to measure CPU usage and memory usage during evaluation
def evaluate_sklearn_random_forest(n_estimators):   
    start_time = time.time()
    rf_classifier = RandomForestClassifier(n_estimators)
    rf_classifier.fit(X_train, y_train)
    rf_prediction = rf_classifier.predict(X_test)
    rf_accuracy = accuracy_score(y_test, rf_prediction)
    rf_runtime = time.time() - start_time
    cpu_usage = measure_cpu()
    return rf_accuracy, rf_runtime, cpu_usage

@measure_memory
def evaluate_custom_random_forest(n_estimators):
    start_time = time.time()
    random_forest = bagging_trees(random_decision_tree, n_estimators)
    random_forest.fit(X_train, y_train)
    rf_prediction = random_forest.predict(X_test)
    rf_accuracy = accuracy_score(y_test, rf_prediction)
    rf_runtime = time.time() - start_time
    cpu_usage = measure_cpu()
    return rf_accuracy, rf_runtime, cpu_usage


In [ ]:
def generate_dataset(size):
    X = np.random.rand(size, 1)
    y = np.random.randint(0, 2, size=size)  # Generating random labels
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    return X_train, X_test, y_train, y_test

In [ ]:
dataset_sizes = [10, 100, 1000, 10000, 100000]
iterations = [10, 100, 1000, 10000]

memory_usages = []
runtimes = []
cpu_usage = []
accuracies = []

memory_usages_custom = []
runtimes_custom = []
cpu_usage_custom = []
accuracies_custom = []

In [ ]:
total_iterations = len(iterations)
progress_bar = tqdm(total=total_iterations)
size = 1000
for n_estimators in iterations:
    progress_bar.update(1)

    results, memory = evaluate_sklearn_random_forest(n_estimators)
    accuracy, runtime, cpu = results
    memory_usages.append((size, n_estimators, memory))
    runtimes.append((size, n_estimators, runtime))
    cpu_usage.append((size, n_estimators, cpu))
    accuracies.append((size, n_estimators, accuracy))

In [ ]:
total_iterations = len(iterations)
progress_bar = tqdm(total=total_iterations)
size = 1000
for n_estimators in iterations:
    progress_bar.update(1)

    results, memory = evaluate_custom_random_forest(n_estimators)
    accuracy, runtime, cpu = results
    memory_usages_custom.append((size, n_estimators, memory))
    runtimes_custom.append((size, n_estimators, runtime))
    cpu_usage_custom.append((size, n_estimators, cpu))
    accuracies_custom.append((size, n_estimators, accuracy))

In [ ]:
plt.figure(figsize=(10, 6))

# Plot Custom RF accuracy comparison
plt.plot([x[1] for x in accuracies_custom], [x[2] for x in accuracies_custom], marker='o', linestyle='-', color='blue', label='Custom RF')

# Plot Scikit-learn RF accuracy comparison
plt.plot([x[1] for x in accuracies], [x[2] for x in accuracies], marker='o', linestyle='-', color='red', label='Scikit-Learn RF')

plt.title('Accuracy Comparison RF Implementations')
plt.xlabel('Iterations')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))

# Custom RF memory usage
plt.plot([x[1] for x in memory_usages_custom], [x[2] for x in memory_usages_custom], marker='o', linestyle='-', color='blue', label='Custom RF')

# Scikit-learn RF memory usage
plt.plot([x[1] for x in memory_usages], [x[2] for x in memory_usages], marker='o', linestyle='-', color='red', label='Scikit-learn RF')

plt.title('Memory Comparison RF Implementations')
plt.xlabel('Iterations')
plt.ylabel('Memory (MB)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))

# Plot Scikit-learn RF runtime comparison
plt.plot([x[1] for x in runtimes], [x[2] for x in runtimes], marker='o', linestyle='-', color='blue', label='Scikit-Learn RF')

# Plot Custom RF runtime comparison
plt.plot([x[1] for x in runtimes_custom], [x[2] for x in runtimes_custom], marker='o', linestyle='-', color='red', label='Custom RF')

plt.title('Runtime Comparison RF Implementations')
plt.xlabel('Iterations')
plt.ylabel('Runtime')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))

# Plot Scikit-learn RF CPU usage comparison
plt.plot([x[1] for x in cpu_usage], [x[2] for x in cpu_usage], marker='o', linestyle='-', color='blue', label='Scikit-Learn RF')

# Plot Custom RF CPU usage comparison
plt.plot([x[1] for x in cpu_usage_custom], [x[2] for x in cpu_usage_custom], marker='o', linestyle='-', color='red', label='Custom RF')

plt.title('CPU Usage Comparison RF Implementations')
plt.xlabel('Iterations')
plt.ylabel('CPU Usage (%)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
memory_usages = []
runtimes = []
cpu_usage = []
accuracies = []

memory_usages_custom = []
runtimes_custom = []
cpu_usage_custom = []
accuracies_custom = []

total_iterations = len(dataset_sizes)
progress_bar = tqdm(total=total_iterations)

for size in dataset_sizes:
    progress_bar.update(1)

    results, memory = evaluate_sklearn_random_forest(100)
    accuracy, runtime, cpu = results
    memory_usages.append((size, n_estimators, memory))
    runtimes.append((size, n_estimators, runtime))
    cpu_usage.append((size, n_estimators, cpu))
    accuracies.append((size, n_estimators, accuracy))

total_iterations = len(dataset_sizes)
progress_bar = tqdm(total=total_iterations)

for size in dataset_sizes:
    progress_bar.update(1)

    results, memory = evaluate_custom_random_forest(100)
    accuracy, runtime, cpu = results
    memory_usages_custom.append((size, n_estimators, memory))
    runtimes_custom.append((size, n_estimators, runtime))
    cpu_usage_custom.append((size, n_estimators, cpu))
    accuracies_custom.append((size, n_estimators, accuracy))

# Accuracy comparison
plt.figure(figsize=(10, 6))
plt.plot([x[0] for x in accuracies_custom], [x[2] for x in accuracies_custom], marker='o', linestyle='-', color='blue', label='Custom RF')
plt.plot([x[0] for x in accuracies], [x[2] for x in accuracies], marker='o', linestyle='-', color='red', label='Scikit-Learn RF')
plt.title('Accuracy Comparison RF Implementations')
plt.xlabel('Dataset Sizes')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# Memory comparison
plt.figure(figsize=(10, 6))
plt.plot([x[0] for x in memory_usages_custom], [x[2] for x in memory_usages_custom], marker='o', linestyle='-', color='blue', label='Custom RF')
plt.plot([x[0] for x in memory_usages], [x[2] for x in memory_usages], marker='o', linestyle='-', color='red', label='Scikit-Learn RF')
plt.title('Memory Comparison RF Implementations')
plt.xlabel('Dataset Sizes')
plt.ylabel('Memory (MB)')
plt.legend()
plt.grid(True)
plt.show()

# Runtime comparison
plt.figure(figsize=(10, 6))
plt.plot([x[0] for x in runtimes], [x[2] for x in runtimes], marker='o', linestyle='-', color='blue', label='Scikit-Learn RF')
plt.plot([x[0] for x in runtimes_custom], [x[2] for x in runtimes_custom], marker='o', linestyle='-', color='red', label='Custom RF')
plt.title('Runtime Comparison RF Implementations')
plt.xlabel('Dataset Sizes')
plt.ylabel('Runtime')
plt.legend()
plt.grid(True)
plt.show()

# CPU usage comparison
plt.figure(figsize=(10, 6))
plt.plot([x[0] for x in cpu_usage], [x[2] for x in cpu_usage], marker='o', linestyle='-', color='blue', label='Scikit-Learn RF')
plt.plot([x[0] for x in cpu_usage_custom], [x[2] for x in cpu_usage_custom], marker='o', linestyle='-', color='red', label='Custom RF')
plt.title('CPU Usage Comparison RF Implementations')
plt.xlabel('Dataset Sizes')
plt.ylabel('CPU Usage (%)')
plt.legend()
plt.grid(True)
plt.show()